In [1]:
import nltk #need for dealing with text
import os # need for looping through folders
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math # need for computing TF-IDF score

In [2]:
pip install num2words

Note: you may need to restart the kernel to use updated packages.


In [3]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

In [4]:
title = "used_files"
os.chdir("C:/mini_newsgroups/comp.graphics/")
paths= []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'): 
    for i in filenames: 
        paths.append(str(dirpath)+str("\\")+i)
        
print(dirpath)
paths[0]
print(len(paths))

C:\mini_newsgroups\comp.graphics/used_files/
4


In [5]:
myfile = open("C:/mini_newsgroups/comp.graphics/used_files/37916")
txt = myfile.read()
print(txt)
myfile.close()

Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!asuvax!cs.utexas.edu!zaphod.mps.ohio-state.edu!saimiri.primate.wisc.edu!usenet.coe.montana.edu!news.u.washington.edu!uw-beaver!cs.ubc.ca!unixg.ubc.ca!kakwa.ucs.ualberta.ca!ersys!joth
From: joth@ersys.edmonton.ab.ca (Joe Tham)
Newsgroups: comp.graphics
Subject: Where can I find SIPP?
Message-ID: <yFXJ2B2w165w@ersys.edmonton.ab.ca>
Date: Mon, 05 Apr 93 14:58:21 MDT
Organization: Edmonton Remote Systems #2, Edmonton, AB, Canada
Lines: 11

        I recently got a file describing a library of rendering routines 
called SIPP (SImple Polygon Processor).  Could anyone tell me where I can 
FTP the source code and which is the newest version around?
        Also, I've never used Renderman so I was wondering if Renderman 
is like SIPP?  ie. a library of rendering routines which one uses to make 
a program that creates the image...

                                        Thanks,  Joe Tham

--
Jo

In [6]:
def print_doc(id):
    file = open(dirpath+'/'+ filenames[id],'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)
    
print_doc(0)
print_doc(1)

Path: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!asuvax!cs.utexas.edu!zaphod.mps.ohio-state.edu!saimiri.primate.wisc.edu!usenet.coe.montana.edu!news.u.washington.edu!uw-beaver!cs.ubc.ca!unixg.ubc.ca!kakwa.ucs.ualberta.ca!ersys!joth
From: joth@ersys.edmonton.ab.ca (Joe Tham)
Newsgroups: comp.graphics
Subject: Where can I find SIPP?
Message-ID: <yFXJ2B2w165w@ersys.edmonton.ab.ca>
Date: Mon, 05 Apr 93 14:58:21 MDT
Organization: Edmonton Remote Systems #2, Edmonton, AB, Canada
Lines: 11

        I recently got a file describing a library of rendering routines 
called SIPP (SImple Polygon Processor).  Could anyone tell me where I can 
FTP the source code and which is the newest version around?
        Also, I've never used Renderman so I was wondering if Renderman 
is like SIPP?  ie. a library of rendering routines which one uses to make 
a program that creates the image...

                                        Thanks,  Joe Tham

--
Jo

In [7]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

def stemming(data):
    stemmer= PorterStemmer()
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

def convert_lower_case(data):
    return np.char.lower(data)

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], '')
        data = np.char.replace(data, " ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

In [8]:
def preprocess(data):
    data = remove_header(data)
    data = remove_single_characters(data)
    data = remove_apostrophe(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = convert_lower_case(data)
    data = remove_punctuation(data)
    data = remove_stop_words(data)
    return data

In [9]:
processed_text = []

for i in range(len(filenames)):
    file = open(dirpath+'/'+ filenames[i], 'r', encoding='cp1250',errors='ignore')
    text = file.read().strip()
    file.close()
    
    processed_text.append(word_tokenize(str(preprocess(text))))
    print(processed_text)

[['recent', 'got', 'file', 'describ', 'librari', 'render', 'routin', 'call', 'sipp', 'simpl', 'polygon', 'processor', 'could', 'anyon', 'tell', 'ftp', 'sourc', 'code', 'newest', 'version', 'around', 'also', 'never', 'use', 'renderman', 'wa', 'wonder', 'renderman', 'like', 'sipp', 'ie', 'librari', 'render', 'routin', 'one', 'use', 'make', 'program', 'creat', 'imag', 'thank', 'joe', 'tham', 'joe', 'tham', 'joth', 'ersysedmontonabca']]
[['recent', 'got', 'file', 'describ', 'librari', 'render', 'routin', 'call', 'sipp', 'simpl', 'polygon', 'processor', 'could', 'anyon', 'tell', 'ftp', 'sourc', 'code', 'newest', 'version', 'around', 'also', 'never', 'use', 'renderman', 'wa', 'wonder', 'renderman', 'like', 'sipp', 'ie', 'librari', 'render', 'routin', 'one', 'use', 'make', 'program', 'creat', 'imag', 'thank', 'joe', 'tham', 'joe', 'tham', 'joth', 'ersysedmontonabca'], ['hello', 'everybodi', 'use', 'pixar', 'renderman', 'three', 'scene', 'descript', 'languag', 'creat', 'three', 'world', 'pleas

In [19]:
DF = {}
N= len(processed_text)

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w]= {i}
            
for i in DF:
    DF[i] = len(DF[i])

In [20]:
DF

{'recent': 1,
 'got': 1,
 'file': 1,
 'describ': 1,
 'librari': 2,
 'render': 2,
 'routin': 1,
 'call': 1,
 'sipp': 1,
 'simpl': 2,
 'polygon': 1,
 'processor': 1,
 'could': 2,
 'anyon': 2,
 'tell': 1,
 'ftp': 2,
 'sourc': 1,
 'code': 1,
 'newest': 1,
 'version': 2,
 'around': 1,
 'also': 2,
 'never': 1,
 'use': 2,
 'renderman': 2,
 'wa': 2,
 'wonder': 1,
 'like': 2,
 'ie': 1,
 'one': 2,
 'make': 1,
 'program': 1,
 'creat': 2,
 'imag': 2,
 'thank': 2,
 'joe': 1,
 'tham': 1,
 'joth': 1,
 'ersysedmontonabca': 1,
 'hello': 1,
 'everybodi': 1,
 'pixar': 1,
 'three': 1,
 'scene': 1,
 'descript': 1,
 'languag': 1,
 'world': 1,
 'pleas': 1,
 'help': 1,
 'next': 1,
 'document': 1,
 'nextstep': 1,
 'avail': 2,
 'veri': 2,
 'complic': 1,
 'surfac': 1,
 'shader': 1,
 'bring': 1,
 'life': 1,
 'appli': 1,
 'shadow': 1,
 'reflect': 1,
 'far': 1,
 'understand': 1,
 'defin': 1,
 'environment': 1,
 'map': 1,
 'produc': 1,
 'know': 3,
 'ani': 2,
 'advis': 1,
 'rib': 1,
 'exampl': 1,
 'appreci': 1,
 'adv

In [22]:
total_vocab_size = len(DF)
print(total_vocab_size)

144


In [30]:
#total_vocab =[]
#for x in DF:
#    total_vocab.append(x)


In [31]:
total_vocab =[x for x in DF]
print(total_vocab[:5])

['recent', 'got', 'file', 'describ', 'librari']


In [35]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c
word ="know"
print("(word) : ",word," (Frequency) : ",doc_freq(word))

(word) :  know  (Frequency) :  3


tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])
    
    for token in np.unique(tokens):
        
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        
        tf_idf[doc, token] = tf*idf
        
    doc += 1

def matching_score(k, query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("Matching Score")
    print("\nQuery:", query)
    print("")
    print(tokens)
    
    query_weights = {}
    for key in tf_idf:
        
        if key[1] in tokens:
            try:
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
                
    query_weights = sorted(query_weights.items(),
                           key=lambda x: x[1],
                           reverse=True)
    
    print("")
    
    l = []
    
    for i in query_weights[:k]:
        l.append(i[0])
        
    print(l)
    
matching_score(2,"I recently got a file describing a library")